In [1]:
pip install vaderSentiment
pip install clean-text

Note: you may need to restart the kernel to use updated packages.


In [74]:
import string 
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib. cm as cm
import re

import seaborn as sns
import matplotlib.pyplot as plt 


import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from cleantext import clean

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
from keras.layers import LSTM, GRU
from keras.layers import Dense, Dropout
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

import matplotlib
import matplotlib.pyplot as plt

In [129]:
def removePunctTokens(tokens):
    punctuation = set(string.punctuation) 
    newTokens = []
    for token in tokens:
        if token in punctuation:
            continue
        else:
            newTokens.append(token)
            
def replaceWithSpaces(string, characters):
    puncutation = set(characters)
    outString = string
    for character in characters:
        outString = outString.replace(character, " ")
        
    return outString

def removeHtml(string):
    outstring = re.sub(f"<a [^<]*>","",string)
    outstring = re.sub(f"<\/a>", "",outstring)
    outstring = outstring.replace("</a>", "")
    outstring = outstring.replace("<br />", "")
    return outstring

def removeLink(string):
    return re.sub(f"https?:\/\/(.*?[\s+]|.*?$)"," ",string)

def removeAts(string):
    return re.sub(f"@\w*"," ",string)

def removeRetweet(string):
    return re.sub(f"^rt *:","",string)

def cleanTweet(string):
    return removeLink(removeRetweet(removeAts(string)))

In [4]:
def sent2vec(sentence, embeddings):
    tokens = nltk.word_tokenize(sentence.lower())
    cleaned_tokens = cleanList(tokens) #unclear what "special character" are. I'm cleaning tokens of any non-alphabetic characters
    word_embeddings = []
    for token in cleaned_tokens:
        if token in embeddings:
            word_embeddings.append(embeddings[token])
        else:
            word_embeddings.append(np.zeros(300))
    emb_arr = np.array(word_embeddings)
    norm_vec = emb_arr.sum(axis=0)
    if type(norm_vec) != np.ndarray:
         return np.zeros(300)
    return norm_vec / np.sqrt((norm_vec ** 2).sum())

def removeNonAlphabetic(string):
    return re.sub("[\W\d]","",string)


def cleanTokens(tokens):
    output = []
    for token in tokens:
        cleanToken = removeNonAlphabetic(token)
        if len(cleanToken) == 0:
            continue
        output.append(cleanToken)
    return output

def removeStopWords(tokens):
    stop_words = set(stopwords.words('english'))
    cleanList = []
    for token in tokens:
        if token not in stop_words:
            cleanList.append(token)
    return cleanList

def cleanList(tokens):
    return removeStopWords(cleanTokens(tokens))

def plot_history(history):
    fig, ax_lst = plt.subplots(1, 2, sharex=True, sharey=True)
    
    epochs = np.array(history.epoch) + 1
    
    ax_lst[0].plot(epochs, history.history['loss'], label='Training')    
    ax_lst[0].plot(epochs, history.history['val_loss'], label='Testing')
    ax_lst[0].set_ylabel('Loss')
    ax_lst[0].set_xlabel('Epoch')
    ax_lst[0].set_xticks(epochs)
    
    ax_lst[1].plot(epochs, history.history['accuracy'], label='Training')    
    ax_lst[1].plot(epochs, history.history['val_accuracy'], label='Testing')
    ax_lst[1].set_ylabel('Accuracy')
    ax_lst[1].set_xlabel('Epoch')
    ax_lst[1].set_xticks(epochs)
    
    fig.tight_layout()

In [5]:
# Testing out the pre-processed twitter tweets success rates, neutral bounds -.1 and .1

In [6]:
negative_df = pd.read_csv("processedNegative.csv")
neutral_df = pd.read_csv("processedNegative.csv")
positive_df = pd.read_csv("processedPositive.csv")

print(negative_df.shape)
print(neutral_df.shape)
print(positive_df.shape)

(0, 1117)
(0, 1117)
(0, 1186)


In [7]:
sentiment = SentimentIntensityAnalyzer()
numNegative = 0
for (columnName, columnData) in negative_df.iteritems():
    scores = sentiment.polarity_scores(columnName)
    if(scores['compound'] < -.1):
        numNegative = numNegative + 1
print(numNegative / 1117)

0.612354521038496


In [8]:
sentiment = SentimentIntensityAnalyzer()
numNeutral = 0
for (columnName, columnData) in neutral_df.iteritems():
    scores = sentiment.polarity_scores(columnName)
    if(scores['compound'] > -.1 and scores['compound'] < .1):
        numNeutral = numNeutral + 1
print(numNeutral / 1117)

0.1638316920322292


In [9]:
sentiment = SentimentIntensityAnalyzer()
numPositive = 0
for (columnName, columnData) in positive_df.iteritems():
    scores = sentiment.polarity_scores(columnName)
    if(scores['compound'] > .1):
        numPositive = numPositive + 1
print(numPositive / 1117)

0.8782452999104745


In [10]:
# Testing out Sentiment 140 dataset, neutral bounds, only positive and negative tweets

In [11]:
sent_df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding ="ISO-8859-1", names=['score','id','date','query','specialOne','data'] )
sent_df.head()

,score,id,date,query,specialOne,data
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
sent_df[['score','data']].iloc[800000][0] == 4

True

In [13]:
positive140List = []
negative140List = []

for index, row in sent_df[['score','data']].iterrows():
    if row[0] == 0:
        negative140List.append(removeLink(row[1]))
    elif row[0] == 4:
        positive140List.append(removeLink(row[1]))



In [14]:
print(len(negative140List))
print(len(positive140List))

800000
800000


In [15]:
sentiment = SentimentIntensityAnalyzer()
numPos = 0
numNeg = 0
for index, row in sent_df[['score','data']].iterrows():
    scores = sentiment.polarity_scores(row[1])
    if(scores['compound'] > 0 and row[0] == 4):
        numPos = numPos + 1
    if(scores['compound'] <= 0 and row[0] == 0 ):
        numNeg = numNeg + 1
print("Positive accuracy: ", numPos / len(positive140List))
print("Negative accuracy: ", numNeg / len(negative140List))

Positive accuracy:  0.62628375
Negative accuracy:  0.67881125


In [16]:
sentiment.polarity_scores(negative140List[2])

{'neg': 0.0, 'neu': 0.842, 'pos': 0.158, 'compound': 0.4939}

In [17]:
# Testing out Testing out twitter sentiment analysis dataset, neutral bounds -.1 and .1

In [18]:
twitter_df = pd.read_csv("twitter_training.csv", names=['id','topic','sentiment','data'] )
twitter_df.drop(twitter_df[pd.isna(twitter_df['data'])].index,inplace = True)

In [19]:
twitter_df.head()
#this dataset contains duplicates phrased differently, which will give us more data on how well vader can handle alternate phrasing

,id,topic,sentiment,data
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [20]:
positiveList = []
negativeList = []
neutralList = []

for index, row in twitter_df[['sentiment','data']].iterrows():
    if row[0] == 'Positive':
        positiveList.append(row[1])
    elif row[0] == 'Negative':
        negativeList.append(row[1])
    elif row[0] == 'Neutral':
        neutralList.append(row[1])
    

In [21]:
print(len(negativeList))
print(len(positiveList))
print(len(neutralList))

22358
20655
18108


In [22]:
#Only score pos neg with no neutral bounds first - give -0s to neg
sentiment = SentimentIntensityAnalyzer()
numPos = 0
numNeg = 0
for item in negativeList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] <= 0):
        numNeg = numNeg + 1   

for item in positiveList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] > 0):
        numPos = numPos + 1   
        
print("Negative accuracy: ", numNeg / len(negativeList))
print("Positive accuracy: ", numPos / len(positiveList))

Negative accuracy:  0.7354414527238572
Positive accuracy:  0.6761074800290486


In [23]:
#Only score pos neg with no neutral bounds first - give 0s to pos
sentiment = SentimentIntensityAnalyzer()
numPos = 0
numNeg = 0
for item in negativeList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] < 0):
        numNeg = numNeg + 1   

for item in positiveList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] >= 0):
        numPos = numPos + 1   
        
print("Negative accuracy: ", numNeg / len(negativeList))
print("Positive accuracy: ", numPos / len(positiveList))

Negative accuracy:  0.5818946238482869
Positive accuracy:  0.8254659888646817


In [24]:
#Only score pos neg with .05 neutral bounds
sentiment = SentimentIntensityAnalyzer()
numPos = 0
numNeg = 0
numNeu = 0
for item in negativeList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] < -.05):
        numNeg = numNeg + 1   

for item in positiveList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] > .05):
        numPos = numPos + 1   
        
for item in neutralList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] >= -.05 and scores['compound'] <= .05):
        numNeu = numNeu + 1   

print("Negative accuracy: ", numNeg / len(negativeList))
print("Positive accuracy: ", numPos / len(positiveList))
print("Neutral accuracy: ", numNeu / len(neutralList))

Negative accuracy:  0.5736201806959478
Positive accuracy:  0.6714112805616074
Neutral accuracy:  0.2109564833222885


In [25]:
#Only score pos neg with .1 neutral bounds
sentiment = SentimentIntensityAnalyzer()
numPos = 0
numNeg = 0
numNeu = 0
for item in negativeList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] < -.05):
        numNeg = numNeg + 1   

for item in positiveList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] > .05):
        numPos = numPos + 1   
        
for item in neutralList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] >= -.1 and scores['compound'] <= .1):
        numNeu = numNeu + 1   

print("Negative accuracy: ", numNeg / len(negativeList))
print("Positive accuracy: ", numPos / len(positiveList))
print("Neutral accuracy: ", numNeu / len(neutralList))

Negative accuracy:  0.5736201806959478
Positive accuracy:  0.6714112805616074
Neutral accuracy:  0.22978793903247183


In [28]:
#carblac dataset https://huggingface.co/datasets/carblacac/twitter-sentiment-analysis
testTweets = open('test_62k.txt', 'r', encoding='utf-8')
Lines = testTweets.readlines()

negCarList = []
posCarList = []
for line in Lines:
    values = line.split("\t")
    if values[0] == "1":
        posCarList.append(values[1])
    else:
        negCarList.append(values[1])
print("num neg: ", len(negCarList))
print("num pos: ", len(posCarList))

num neg:  30969
num pos:  31029


In [30]:
identifiedPos = 0
identifiedNeg = 0

for item in negCarList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] < 0):
        identifiedNeg = identifiedNeg + 1   

for item in posCarList:
    scores = sentiment.polarity_scores(item)
    if(scores['compound'] > 0):
        identifiedPos = identifiedPos + 1
        
print("Neg accuracy: ", identifiedNeg / len(negCarList))
print("Pos accuracy: ", identifiedPos / len(posCarList))

Neg accuracy:  0.4336917562724014
Pos accuracy:  0.6244158690257501


In [31]:
#Things look bad - let's make a NN
gloveFile = open('glove.6B.300d.txt', 'r', encoding='utf-8')

embeddings_index = {}
for line in gloveFile.readlines():
    values = line.split()
    embeddings_index[values[0]] = [float(value) for value in values[1:]]

In [32]:
testTweets = open('train_150k.txt', 'r', encoding='utf-8')
Lines = testTweets.readlines()
cleanTweets = []
scores = []

for line in Lines:
    values = line.split("\t")
    scores.append(int(values[0]))
    cleanTweets.append(cleanTweet(values[1]))


tweetGlove = np.array([sent2vec(tweet,embeddings_index) for tweet in cleanTweets])
tweetGlove = np.nan_to_num(tweetGlove)  

npScores = np.array(scores)


C:\Users\Seth\AppData\Local\Temp/ipykernel_4940/704902115.py:14: RuntimeWarning: invalid value encountered in true_divide
  return norm_vec / np.sqrt((norm_vec ** 2).sum())


In [33]:
x_train, x_test, y_train, y_test = train_test_split(tweetGlove,npScores,test_size=.2,random_state=33)

In [34]:
model = Sequential()
model.add(Dense(64, input_dim=300, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

history = model.fit(x_train, y_train,
                   validation_data=(x_test, y_test),
                   epochs=20,
                   batch_size=25,
                   verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                19264     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 21,889
Trainable params: 21,889
Non-traina

In [35]:
#embed training set

In [36]:
testTweets = open('test_62k.txt', 'r', encoding='utf-8')
Lines = testTweets.readlines()
testScores = []
cleanTestTweets= []

for line in Lines:
    values = line.split("\t")
    testScores.append(int(values[0]))
    cleanTestTweets.append(cleanTweet(values[1]))

testGlove = np.array([sent2vec(tweet,embeddings_index) for tweet in cleanTestTweets])
testGlove = np.nan_to_num(testGlove)  
print(testGlove.shape)
prediction = model.predict(testGlove)
counter = 0
correctGuesses = 0
for score in testScores:
    if score == 1 and prediction[counter] > .5:
        correctGuesses = correctGuesses + 1
    elif score == 0 and prediction[counter] <= .5:
        correctGuesses = correctGuesses + 1
    counter = counter + 1

print(correctGuesses/len(testScores))

C:\Users\Seth\AppData\Local\Temp/ipykernel_4940/704902115.py:14: RuntimeWarning: invalid value encountered in true_divide
  return norm_vec / np.sqrt((norm_vec ** 2).sum())


(61998, 300)
1938/1938 [==============================] - 1s 468us/step
0.7381689731926836


In [37]:
prediction = model.predict(testGlove)

1938/1938 [==============================] - 1s 502us/step


In [38]:
prediction[0] > .5

array([ True])

In [51]:
sent_df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding ="ISO-8859-1", names=['score','id','date','query','specialOne','data'] )

testTweets = open('train_150k.txt', 'r', encoding='utf-8')
Lines = testTweets.readlines()

compiledTweets = []
compiledScores = []

#Append 160000 processed no emoticon
for index, row in sent_df[['score','data']].iterrows():
    if row[0] == 0:
        compiledTweets.append(cleanTweet(row[1]))
        compiledScores.append(0)
    elif row[0] == 4:
        compiledTweets.append(cleanTweet(row[1]))
        compiledScores.append(1)

#append cbac train tweets  
for line in Lines:
    values = line.split("\t")
    if values[0] == "1":
        compiledTweets.append(cleanTweet(values[1]))
        compiledScores.append(1)
    else:
        compiledTweets.append(cleanTweet(values[1]))
        compiledScores.append(0)

In [52]:
print(len(compiledTweets))
print(len(compiledScores))

1749985
1749985


In [41]:
#Things look bad - let's make a NN
gloveFile = open('glove.6B.300d.txt', 'r', encoding='utf-8')

embeddings_index = {}
for line in gloveFile.readlines():
    values = line.split()
    embeddings_index[values[0]] = [float(value) for value in values[1:]]

In [42]:
tweetGlove = np.array([sent2vec(tweet,embeddings_index) for tweet in compiledTweets])
tweetGlove = np.nan_to_num(tweetGlove)  

npCompiledScores = np.array(compiledScores)

C:\Users\Seth\AppData\Local\Temp/ipykernel_4940/704902115.py:14: RuntimeWarning: invalid value encountered in true_divide
  return norm_vec / np.sqrt((norm_vec ** 2).sum())


In [43]:
x_train, x_test, y_train, y_test = train_test_split(tweetGlove,npCompiledScores,test_size=.2,random_state=33)

In [44]:
model = Sequential()
model.add(Dense(128, input_dim=300, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

history = model.fit(x_train, y_train,
                   validation_data=(x_test, y_test),
                   epochs=20,
                   batch_size=25,
                   verbose=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               38528     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_7 (Dense)             (None, 16)               

In [53]:
#taking every dataset, pos neg only, embed them, 0 negative, 1 positive
negative_df = pd.read_csv("processedNegative.csv") #exclude
positive_df = pd.read_csv("processedPositive.csv") #exclude

twitter_df = pd.read_csv("twitter_training.csv", names=['id','topic','sentiment','data'] )#exclude
twitter_df.drop(twitter_df[pd.isna(twitter_df['data'])].index,inplace = True) #exclude


for (columnName, columnData) in positive_df.iteritems():
    compiledTweets.append(cleanTweet(columnName))
    compiledScores.append(1)
    
for (columnName, columnData) in negative_df.iteritems():
    compiledTweets.append(cleanTweet(columnName))
    compiledScores.append(0)

for index, row in twitter_df[['sentiment','data']].iterrows():
    if row[0] == 'Positive':
        compiledTweets.append(cleanTweet(row[1]))
        compiledScores.append(1)
    elif row[0] == 'Negative':
        compiledTweets.append(cleanTweet(row[1]))
        compiledScores.append(0)

In [54]:
tweetGlove = np.array([sent2vec(tweet,embeddings_index) for tweet in compiledTweets])
tweetGlove = np.nan_to_num(tweetGlove)  

npCompiledScores = np.array(compiledScores)

C:\Users\Seth\AppData\Local\Temp/ipykernel_4940/704902115.py:14: RuntimeWarning: invalid value encountered in true_divide
  return norm_vec / np.sqrt((norm_vec ** 2).sum())


In [55]:
x_train, x_test, y_train, y_test = train_test_split(tweetGlove,npCompiledScores,test_size=.2,random_state=33)

In [56]:
model = Sequential()
model.add(Dense(128, input_dim=300, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

history = model.fit(x_train, y_train,
                   validation_data=(x_test, y_test),
                   epochs=20,
                   batch_size=25,
                   verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               38528     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 32)                2080      
                                                                 
 dropout_7 (Dropout)         (None, 32)                0         
                                                                 
 dense_12 (Dense)            (None, 16)               

In [57]:
model.save_weights('17MilTweetNN')


In [58]:
model.load_weights('17MilTweetNN')

In [61]:
values = model.predict(x_test)

11221/11221 [==============================] - 19s 2ms/step


In [62]:
values[0]

array([0.16694458], dtype=float32)

In [67]:
dino_df = pd.read_csv("RyanData/dino_tweets_15k.csv")
lol_df = pd.read_csv("RyanData/league_tweets_15k.csv")
musk_df = pd.read_csv("RyanData/musk_tweets_15k.csv")
dino_df['text'][24]

0     RT @purinkoo: jungkookie is really the pretty ...
1     Light Painting 101: Illuminating a terrifying ...
2     Light Painting 101: Illuminating a terrifying ...
3     RT @Coolio_Art: Small art tip for drawing rept...
4     RT @nahi_tigray: There is never true peace wit...
5     RT @Coolio_Art: Small art tip for drawing rept...
6     RT @purinkoo: jungkookie is really the pretty ...
7     RT @Coolio_Art: Small art tip for drawing rept...
8     RT @mulu96: @hrw There is never true peace wit...
9     @JRPTO @esjyt11 @Texan81 @Lauren_Southern And ...
10    RT @JurassicAddict: Man this year really felt ...
11    A fossil unearthed on Koh Por island in Koh Ko...
12       @gendernudetral Ooooooh its the dinosaur thing
13    RT @realrkwest: @ThatEricAlper Teletubbies, Ba...
14    RT @Coolio_Art: Small art tip for drawing rept...
15    RT @mulu96: @hrw There is never true peace wit...
16    RT @mulu96: @hrw There is never true peace wit...
17                  @_krystoast The dinosaur! Ra

In [133]:
dino_df['cleanText'] = [cleanTweet(clean(text,no_emoji=True)) for text in dino_df['text']]

In [134]:
dinoGlove = np.array([sent2vec(tweet,embeddings_index) for tweet in dino_df['cleanText']])
dinoGlove = np.nan_to_num(dinoGlove)  

C:\Users\Seth\AppData\Local\Temp/ipykernel_4940/704902115.py:14: RuntimeWarning: invalid value encountered in true_divide
  return norm_vec / np.sqrt((norm_vec ** 2).sum())


In [135]:
lol_df['cleanText'] = [cleanTweet(clean(text,no_emoji=True)) for text in lol_df['text']]

In [136]:
lolGlove = np.array([sent2vec(tweet,embeddings_index) for tweet in lol_df['cleanText']])
lolGlove = np.nan_to_num(lolGlove) 

In [137]:
musk_df['cleanText'] = [cleanTweet(clean(text,no_emoji=True)) for text in musk_df['text']]

In [138]:
muskGlove = np.array([sent2vec(tweet,embeddings_index) for tweet in musk_df['cleanText']])
muskGlove = np.nan_to_num(muskGlove) 

In [141]:
dinoResults = model.predict(dinoGlove)
lolResults = model.predict(lolGlove)
muskResults = model.predict(muskGlove)

472/472 [==============================] - 1s 2ms/step


In [153]:
def scoreValue(value):
    if value >= .5:
        return True
    else:
        return False
    
def getNumPositive(results):
    counter = 0
    for result in results:
        if scoreValue(result):
            counter = counter + 1
    return counter

In [154]:
posDino = getNumPositive(dinoResults)
posLol = getNumPositive(lolResults)
posMusk = getNumPositive(muskResults)

print(posDino / len(dinoGlove))
print(posLol / len(lolGlove))
print(posMusk / len(muskGlove))

0.6749554778708529
0.8742084432717678
0.6058113307682101
